# Indifference risk attitude

In this notebook, we will introduce useful concept of **indifference risk attitude** ($\eta^*$). Examining how $\eta^*$ changes as a function of presented gamble pairs and agent's wealth can give useful intuitions about the choice preferences of isoelastic agents.

In [ ]:
from utils.models import IsoelasticWealthChange, Gamble, IsoelasticAgent, find_indifference_eta
from utils.plotting import aligned_imshow_cbar
from utils.style import rc_style, eta_dynamic_color

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from matplotlib.colors import LinearSegmentedColormap, to_rgba
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

mpl.rcParams.update(rc_style)
mpl.rcParams.update({"axes.spines.right": True, "axes.spines.top": True})

## Choice preference

Let's imagine that we have a population of isoelastic agents. All agents use isoelastic utility function to guide their choice. However, agents differ in terms of their risk preference: some agents are risk-seeking with $\eta < 0$, some are risk neutral $\eta \approx 0$ and others are risk-averse $\eta > 0$. Let's assume that we endow all agents with the same amount of wealth ($x_0$). Now we choose a wealth dynamic parameter and construct two gambles. Let's call them left gamble and right gamble. Both gambles consists of two deterministic wealth changes that agents know in advance. This means that agents are able to compute their expected wealths after choosing either gamble. Both wealth changes constituting a gamble have the same probabilities of being realized after a gamble is selected. 

Now an agent is asked to make a choice – choose left or right gamble. In some cases all agents would agree to choose one gamble over the other. However, for some gambles part of the population would choose left gamble, and the other part would choose right gamble. This observation leads to an interesting question – *What is the risk attitude value for which preference changes?* In other words we will try to find a $\eta^*$ such that an agent using isoelastic function $u(\eta^*, x)$ would be **indifferent** to the choice of one of two gambles. The $\eta^*$ acts as a tipping point for agent's preference – agents with risk preference $\eta<\eta^*$ would always prefer "riskier" gamble, whereas agents with $\eta>\eta^*$ would always select a "safer" option. The other advantage of inspecting $\eta^*$ for a given gamble is the discriminability between agents. If agents are deterministic, and we want to distinguish between two agents, $\eta_1$ and $\eta_2>\eta_1$, based on observed choice it is sufficient to let them play a gamble with $\eta_1<\eta^*<\eta_2$. For example, if we want to distinguish risk-neutral agent with $u(x)=x$ from risk-averse agent with $u(x)=log(x)$ we need to create gamble for which $\eta^*\in (0, 1)$.

Figure below shows difference in expected utility change between left and right gamble depending on agent's risk attitude $\eta$. Blue area under curve represents range of risk attitude for which agents would choose left gamble. Conversely, pink area shows $\eta$ range for which right gamble is preferred. The root of the function (place where it intersects with x-axis is the indifference risk preference $\eta^*$). Sliders allow to modify all four growth rates constituting both gambles (`gamma_l1` and `gamma_l2` are growth rates for the left gamble, `gamma_r1` and `gamma_r2` are growth rates for the right gamble). `eta_dynamic` slider controls wealth dynamics, and `x0` controls initial wealth.

In [ ]:
def plot_gamble_edu_agents(gamma_l1, gamma_l2, gamma_r1, gamma_r2, 
                           eta_dynamic=0, eta_agent_max=1, x0=10):
    """Gamble preference for a gamble pair depending on agent's risk aversion.
    
    Visualize preference towards one of the gambles from a pair (trial) 
    depending on isoelastic agent's utility function, i.e., risk preference. 
    Indifference eta is the value of agent's risk attitude for which an agent is
    indifferent between choosing left or right gamble. It is a property of a 
    gamble but it can also depend on agent's wealth.
    
    Args:
        gamma_l1 (float):
            Growth rate for the first wealth change in the left gamble.
        gamma_l2 (float):
            Growth rate for the second wealth change in the left gamble.
        gamma_r1 (float):
            Growth rate for the first wealth change in the right gamble.
        gamma_r2 (float):
            Growth rate for the second wealth change in the right gamble.
        eta_agent_max (float):
            Determines range of agent's risk attitude for which preference is 
            calculated.
        eta_dynamic (float):
            Wealth dynamic parameter.
        x0 (float):
            Initial wealth.
    """
    eta_agent_min = -eta_agent_max
    col_l = "cornflowerblue"
    col_r = "orchid"
    
    gambles = [
        Gamble(
            wc1=IsoelasticWealthChange(g1, eta_dynamic),
            wc2=IsoelasticWealthChange(g2, eta_dynamic)
        )
        for g1, g2 in ((gamma_l1, gamma_l2), (gamma_r1, gamma_r2))
    ]

    # Calculate preferences
    etas_agent = np.linspace(eta_agent_min, eta_agent_max, num=100)
    gp_diff = np.zeros(etas_agent.shape)
    for i, eta_agent in enumerate(etas_agent):
        gp_diff[i] = IsoelasticAgent(eta_agent, wealth=x0).gamble_difference(*gambles)

    # Visualize 
    fig, (ax, axp) = plt.subplots(
        figsize=(14, 6), 
        ncols=2, 
        gridspec_kw={'width_ratios': [3, 2]}
    )

    ax.plot(etas_agent, gp_diff, c=eta_dynamic_color(eta_dynamic))
    ax.axhline(0, color="k", linestyle=":")
    ax.set_xlabel(r"agent's $\eta$")
    ax.set_ylabel(r"$\Delta u$ (left $-$ right)")

    # Show trial
    bbox_dict = dict(boxstyle="square", pad=1.25, ec="k", lw=2)
    text_dict_l = dict(va="center", font="Monospace", bbox={**bbox_dict, "fc": col_l})
    text_dict_r = dict(va="center", font="Monospace", bbox={**bbox_dict, "fc": col_r})
    axp.text(.25, .33, f"{gamma_l1:+.3f}", ha="right", **text_dict_l)
    axp.text(.25, .66, f"{gamma_l2:+.3f}", ha="right", **text_dict_l)
    axp.text(.55, .33, f"{gamma_r1:+.3f}", ha="left", **text_dict_r)
    axp.text(.55, .66, f"{gamma_r2:+.3f}", ha="left", **text_dict_r)
    axp.text(.4, .5, r"$+$", ha="center", va="center", fontsize=30)
    axp.set_axis_off()

    roots = np.nonzero(np.diff(np.sign(gp_diff)))[0]
    if len(roots):
        i = roots[0]
        eta_halfstep = np.diff(etas_agent)[0] / 2
        eta_indiff = etas_agent[i] + eta_halfstep
        ax.axvline(eta_indiff, color="k")
    else:
        i = len(etas_agent)

    # Show preferences as colored areas under curve
    if gp_diff[0] >= 0:
        ax.fill_between(etas_agent[:i+1], gp_diff[:i+1], color=col_l)
        ax.fill_between(etas_agent[i:], gp_diff[i:], color=col_r)
    else:
        ax.fill_between(etas_agent[:i+1], gp_diff[:i+1], color=col_r)        
        ax.fill_between(etas_agent[i:], gp_diff[i:], color=col_l)

In [ ]:
eta_dynamic_widget = widgets.FloatSlider(min=-1, max=1, step=0.05, value=0)
gamma_l1_widget = widgets.FloatSlider(min=-100, max=100, step=1, value=3)
gamma_l2_widget = widgets.FloatSlider(min=-100, max=100, step=1, value=2)
gamma_r1_widget = widgets.FloatSlider(min=-100, max=100, step=1, value=9)
gamma_r2_widget = widgets.FloatSlider(min=-100, max=100, step=1, value=-8)

def update_gammas_widget(*args):
    gamma_thr = 10 ** (3 - 3 * eta_dynamic_widget.value) / 10
    for widget in [gamma_l1_widget, gamma_l2_widget, 
                   gamma_r1_widget,  gamma_r2_widget]:
        widget.max = gamma_thr
        widget.min = -gamma_thr
        widget.step = gamma_thr / 100
        widget.value = 0

eta_dynamic_widget.observe(update_gammas_widget, "value")    

interact(
    plot_gamble_edu_agents,
    gamma_l1=gamma_l1_widget,
    gamma_l2=gamma_l2_widget, 
    gamma_r1=gamma_r1_widget, 
    gamma_r2=gamma_r2_widget, 
    eta_dynamic=eta_dynamic_widget, 
    eta_agent_max=widgets.fixed(1), 
    x0=widgets.IntSlider(min=1, max=1000, step=10, value=10)
);

## Gamble space map of $\eta_*$

Now, after we introduced the concept of indifference risk attitude, let's explore how $\eta_*$ changes when we vary growth rates for one gamble from a pair of gambles. For this purpose we will examine 2-dimensional indifference eta heatmaps. 

To better understand this concept let's imagine that we first choose wealth dynamic and then fix the first (left) gamble to have growth rates $\gamma^{\text{ref}}_1$ and $\gamma^{\text{ref}}_2$. Now we can create a choice situation by creating the second (right) gamble with growth rates $\gamma_1$ and $\gamma_2$. For a specified pair of gambles, and agent's wealth $x_0$ we can calulate the value of indifference risk attitude $\eta^*$. This can be done iteratively by finding a root of the $\Delta u$ function (see previous section). We can then visualize $\eta^*$ as a two-parameter function of $\gamma_1$ and $\gamma_2$. In general $\eta_*$ depends on five parameters: $\gamma^{\text{left}}_1$, $\gamma^{\text{left}}_2$, $\gamma^{\text{right}}_1$, $\gamma^{\text{right}}_2$, and $x$. However, if we fix three of them $\gamma^{\text{left}}_1=\gamma^{\text{ref}}_1$, $\gamma^{\text{left}}_2=\gamma^{\text{ref}}_2$, and $x=x_0$, our function become dependent only on $\gamma_1$ and $\gamma_2$, i.e., $\eta^*=\eta^*(\gamma_1, \gamma_2)$.

Figure below shows $\eta^*=\eta^*(\gamma_1, \gamma_2)$. Indifference eta is only calculaled within the range $[-1, 1]$, i.e., values outside of this range are ignored. Blue regions correspond to gambles that when paired with the reference gamble would be preferred by the risk-seeking agents. Violet regions correpond to gambles that would be preferred by the risk-averse agents when paired with the reference gamble. Black cross shows growth rates for the reference (fixed left) gamble. Smaller subplot show how different wealth changes affect initial wealth (solid horizontal line). 

Sliders: 
- `gamma_ref_1`: first growth rate for the left (fixed) gamble 
- `gamma_ref_2`: second growth rate for the left (fixed) gamble 
- `eta_dynamic`: wealth dynamic
- `gamma_sample`: sampling rate for gamble space growth rates (setting this high would significantly increase computation time)
- `x0`: initial wealth

In [ ]:
def plot_indifference_eta_gamble_space(gamma_ref_1, gamma_ref_2, eta_dynamic,
                                       gamma_max, gamma_samples=10, x0=100, 
                                       eta_agent_max=1):
    """Plot indifference eta heatmap."""
    eta_agent_min = -eta_agent_max
    gamma_range = np.linspace(-gamma_max, gamma_max, gamma_samples)
    
    # Create fractals and reference gamble
    fractals = [IsoelasticWealthChange(g, eta=eta_dynamic) for g in gamma_range]
    n_fractals = len(fractals)
    ref_g = Gamble(IsoelasticWealthChange(gamma_ref_1, eta_dynamic),
                   IsoelasticWealthChange(gamma_ref_2, eta_dynamic))

    # Calculate indifference risk attitude
    eta_indiff = np.zeros((n_fractals, n_fractals))
    for i in range(n_fractals):
        for j in range(n_fractals):
            g = Gamble(fractals[i], fractals[j])
            eta_indiff[i, j] = find_indifference_eta(
                g1=g, 
                g2=ref_g,
                x=x0, 
                eta_l=eta_agent_min, 
                eta_r=eta_agent_max, 
                precision=1e-04
            )

    fig, (ax1, ax2) = plt.subplots(
        figsize=(10, 10), 
        nrows=2,
        gridspec_kw={'height_ratios': [3, 1]}
    )

    colors = [to_rgba("cornflowerblue"), (1, 1, 1), to_rgba("orchid")]
    mycmap = LinearSegmentedColormap.from_list("mycmap", colors, N=100)
    
    # Top panel (indifference eta)
    im = ax1.imshow(
        eta_indiff, 
        clim=[eta_agent_min, eta_agent_max], 
        cmap=mycmap, 
        origin="upper"
    )
    aligned_imshow_cbar(ax1, im)

    gamma_mid = (gamma_range[0] + gamma_range[-1]) / 2
    gamma_step = gamma_range[1] - gamma_range[0]

    # Ticks and labels
    ax1.set_xlabel("growth rate $\gamma_1$")
    ax1.set_ylabel("growth rate $\gamma_2$")
    ticks = np.linspace(0, n_fractals - 1, 3)
    tickvalues = (gamma_range[0], gamma_mid, gamma_range[-1])
    ticklabels = [f"{g:.2f}" for g in tickvalues]
    ax1.set_xticks(ticks)
    ax1.set_yticks(ticks)
    ax1.set_xticklabels(ticklabels)
    ax1.set_yticklabels(ticklabels)
    ax1.set_title(f"$\eta^*$ (dynamic={eta_dynamic})")

    # Cross
    cross_x = (gamma_ref_1 + gamma_max) / (2 * gamma_max) * (n_fractals - 1)
    cross_y = (gamma_ref_2 + gamma_max) / (2 * gamma_max) * (n_fractals - 1)
    ax1.text(cross_x, cross_y, r"$\times$", va="center", ha="center", fontsize=40)
    ax1.plot([-.5, n_fractals -.5], [-.5, n_fractals -.5], "k:", alpha=0.3)

    # Bottom panel (wealth change)
    plt.tight_layout()
    pos = ax2.get_position()
    pos.x0 = pos.x0 + .13
    pos.x1 = pos.x1 - .17
    ax2.set_position(pos)

    ax2.plot(
        gamma_range, 
        [f(x0) for f in fractals], 
        ".-", 
        color=eta_dynamic_color(eta_dynamic)
    )
    ax2.axhline(x0, color="k")
    ax2.set_xlim([gamma_range[0] - gamma_step / 2, 
                  gamma_range[-1] + gamma_step / 2])
    ax2.set_xticks(tickvalues)
    ax2.set_ylabel("wealth $x$")
    ax2.set_xlabel("growth rate $\gamma$")

    plt.show()

In [ ]:
eta_dynamic_widget = widgets.FloatSlider(min=-1, max=1, step=0.05, value=0)
gamma_ref_1_widget = widgets.FloatSlider(min=-100, max=100, step=1, value=0)
gamma_ref_2_widget = widgets.FloatSlider(min=-100, max=100, step=1, value=0)
gamma_max_widget = widgets.fixed(100)
x0_widget = widgets.IntSlider(min=1, max=1000, step=10, value=110)

def update_gammas_widget(*args):
    gamma_thr = 10 ** (3 - 3 * eta_dynamic_widget.value) / 10
    for widget in [gamma_ref_1_widget, gamma_ref_2_widget]:
        widget.max = gamma_thr
        widget.min = -gamma_thr
        widget.step = gamma_thr / 100
        widget.value = 0
    gamma_max_widget.value = gamma_thr
    x0_widget.value = 500
    
eta_dynamic_widget.observe(update_gammas_widget, "value")    

interact(
    plot_indifference_eta_gamble_space,
    gamma_ref_1=gamma_ref_1_widget, 
    gamma_ref_2=gamma_ref_2_widget, 
    eta_dynamic=eta_dynamic_widget,
    gamma_max=gamma_max_widget,
    gamma_samples=widgets.IntSlider(min=5, max=100, step=10, value=25),
    x0=x0_widget,
    eta_agent_max=widgets.fixed(1),
);

Few observations/intuitions after examininng $\eta^*$ heatmaps.

> Indifference eta is invariant to $x_0$ only for dynamic given by $\eta=1$. For any other dynamic blue and violet indifference areas shrink with increasing $x_0$. 

> General shape and behavior of indifference areas is invariant to wealth dynamics.

> Its easier to find discriminating gambles when gambles have **similar average growth rates** and **maximally different variance**.